# Initialization

In this section, we just excrtact data from csv file to dataset variable which its type is dictionary with keyes made up with attributes.


Each attribute value is limited here is summary of attributes and their values:


Fiscal Year Released -> [2010, 2013, 2015]


Recidivism Reporting Year -> [2013 2016 2018]


Race - Ethnicity    -> ['Black' 'White']


Age At Release    -> ['<45' '>45']


Convicting Offense Classification     -> ['D Felony' 'Other Felony']
                                    
Convicting Offense Type    -> ['Drug' 'Other' 'Violent']

Convicting Offense Subtype   -> ['Other' 'Trafficking']


Main Supervising District    -> ['3JD' '5JD']

Release Type   -> ['Discharged End of Sentence' 'Parole']
                                    
                                    
Part of Target Population   -> ['No' 'Yes']

Recidivism - Return to Prison numeric  -> [0 1]

Our target feature to predict will be "Recidivism - Return to Prison numeric" so in first part we are going to build a desicion tree using ID3 algorithm to predict this feature. In next part we are going to build a random forest and in last part we use random forest classification in sci kit library to measure our implementation.



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
import random
from pprint import pprint
import numpy as np
import pandas as pd
from math import sqrt
from google.colab import drive
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

drive.mount('/content/drive')
path = '/content/drive/My Drive/IS_HW2/Q2/prison_dataset.csv'
dataset = pd.read_csv(path,
                      names=['Fiscal Year Released','Recidivism Reporting Year','Race - Ethnicity','Age At Release','Convicting Offense Classification',
                                    'Convicting Offense Type','Convicting Offense Subtype','Main Supervising District','Release Type',
                                    'Part of Target Population','Recidivism - Return to Prison numeric'],header = 1)
dataset = shuffle(dataset)

# Entropy Calculations Method


It is commonly used in the construction of decision trees from a training dataset, by evaluating the information gain for each variable, and selecting the variable that maximizes the information gain, which in turn minimizes the entropy and best splits the dataset into groups for effective classification.


IG(S, a) = H(S) – H(S | a)

H(S | a) = sum v in a Sa(v)/S * H(Sa(v))

In our implementation we first define a function to calculate entropy in a column(feature) then we use it in our main function to calculate Information Gain.

In [ ]:
def entropy(target_col):
    """
    Calculate the entropy of a dataset.
    The only parameter of this function is the target_col parameter which specifies the target column
    """
    elements,counts = np.unique(target_col,return_counts = True)
    entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy


def cal_information_gain(data,split_attribute_name,target_name='Recidivism - Return to Prison numeric'):
    """
    Calculate the information gain of a dataset. This function takes three parameters:
    1. data = The dataset for whose feature the IG should be calculated
    2. split_attribute_name = the name of the feature for which the information gain should be calculated
    3. target_name = the name of the target feature. The default for this example is "class"
    """    
    #Calculate the entropy of the total dataset
    total_entropy = entropy(data[target_name])
    
    ##Calculate the entropy of the dataset
    
    #Calculate the values and the corresponding counts for the split attribute 
    vals,counts= np.unique(data[split_attribute_name],return_counts=True)
    
    #Calculate the weighted entropy
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
    
    #Calculate the information gain
    information_gain = total_entropy - Weighted_Entropy
    return information_gain


#ID3 Method


here it's ID3 Pseudocode that we use to implement


ID3 (Examples, Target_Attribute, Attributes)

    Create a root node for the tree
    If all examples are positive, Return the single-node tree Root, with label = +.
    If all examples are negative, Return the single-node tree Root, with label = -.
    If number of predicting attributes is empty, then Return the single node tree Root,
    with label = most common value of the target attribute in the examples.
    Otherwise Begin
        A ← The Attribute that best classifies examples.
        Decision Tree attribute for Root = A.
        For each possible value, vi, of A,
            Add a new tree branch below Root, corresponding to the test A = vi.
            Let Examples(vi) be the subset of examples that have the value vi for A
            If Examples(vi) is empty
                Then below this new branch add a leaf node with label = most common target value in the examples
            Else below this new branch add the subtree ID3 (Examples(vi), Target_Attribute, Attributes – {A})
    End
    Return Root





ID3 Algorithm: This function takes five paramters:

    1. data = the data for which the ID3 algorithm should be run --> In the first run this equals the total dataset

    2. originaldata = This is the original dataset needed to calculate the mode target feature value of the original dataset
    in the case the dataset delivered by the first parameter is empty

    3. features = the feature space of the dataset . This is needed for the recursive call since during the tree growing process
    we have to remove features from our dataset --> Splitting at each node

    4. current_depth: depth of tree to track and stop whenever we reach 3 in depth

    5. max_depth = the limitation of depth (where to stop)
    
    6. target_attribute_name = the name of the target attribute

    7. parent_node_class = This is the value or class of the mode target feature value of the parent node for a specific node. This is 
    also needed for the recursive call since if the splitting leads to a situation that there are no more features left in the feature
    space, we want to return the mode target feature value of the direct parent node.

In [ ]:
def ID3(data,originaldata,features,current_depth,max_depth = 3,target_attribute_name="Recidivism - Return to Prison numeric",parent_node_class = None):
    if(current_depth == max_depth):
        return parent_node_class 
    else:
        current_depth += 1  
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name],return_counts=True)[1])]
        item_values = [cal_information_gain(data,feature,target_attribute_name) for feature in features] #Return the information gain values for the features in the dataset
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]
        tree = {best_feature:{}}
        features = [i for i in features if i != best_feature]
        for value in np.unique(data[best_feature]):
            value = value
            sub_data = data.where(data[best_feature] == value).dropna()
            subtree = ID3(sub_data,dataset,features,current_depth,max_depth,target_attribute_name,parent_node_class)
            tree[best_feature][value] = subtree  
        return(tree) 

#Some Funtions to Predict and Test

In this part to have better perfomance in prediction function I helped from this [link](https://www.python-course.eu/).



Prediction of a new/unseen query instance. This takes two parameters:

    1. The query instance as a dictionary of the shape {"feature_name":feature_value,...}

    2. The tree 


    We do this also in a recursive manner. That is, we wander down the tree and check if we have reached a leaf or if we are still in a sub tree. 
    Since this is a important step to understand, the single steps are extensively commented below.

    1.Check for every feature in the query instance if this feature is existing in the tree.keys() for the first call, 
    tree.keys() only contains the value for the root node 
    --> if this value is not existing, we can not make a prediction and have to 
    return the default value which is the majority value of the target feature

    2. First of all we have to take care of a important fact: Since we train our model with a database A and then show our model
    a unseen query it may happen that the feature values of these query are not existing in our tree model because non of the
    training instances has had such a value for this specific feature. 
    For instance imagine the situation where your model has only seen animals with one to four
    legs - The "legs" node in your model will only have four outgoing branches (from one to four). If you now show your model
    a new instance (animal) which has for the legs feature the vale 5, you have to tell your model what to do in such a 
    situation because otherwise there is no classification possible because in the classification step you try to 
    run down the outgoing branch with the value 5 but there is no such a branch. Hence: Error and no Classification!
    We can address this issue with a classification value of for instance (999) which tells us that there is no classification
    possible or we assign the most frequent target feature value of our dataset used to train the model. Or, in for instance 
    medical application we can return the most worse case - just to make sure... 
    We can also return the most frequent value of the direct parent node. To make a long story short, we have to tell the model 
    what to do in this situation.
    In our example, since we are dealing with animal species where a false classification is not that critical, we will assign
    the value 1 which is the value for the mammal species (for convenience).

    3. Address the key in the tree which fits the value for key --> Note that key == the features in the query. 
    Because we want the tree to predict the value which is hidden under the key value (imagine you have a drawn tree model on 
    the table in front of you and you have a query instance for which you want to predict the target feature 
    - What are you doing? - Correct:
    You start at the root node and wander down the tree comparing your query to the node values. Hence you want to have the
    value which is hidden under the current node. If this is a leaf, perfect, otherwise you wander the tree deeper until you
    get to a leaf node. 
    Though, you want to have this "something" [either leaf or sub_tree] which is hidden under the current node
    and hence we must address the node in the tree which == the key value from our query instance. 
    This is done with tree[keys]. Next you want to run down the branch of this node which is equal to the value given "behind"
    the key value of your query instance e.g. if you find "legs" == to tree.keys() that is, for the first run == the root node.
    You want to run deeper and therefore you have to address the branch at your node whose value is == to the value behind key.
    This is done with query[key] e.g. query[key] == query['legs'] == 0 --> Therewith we run down the branch of the node with the
    value 0. Summarized, in this step we want to address the node which is hidden behind a specific branch of the root node (in the first run)
    this is done with: result = [key][query[key]]

    4. As said in the 2. step, we run down the tree along nodes and branches until we get to a leaf node.
    That is, if result = tree[key][query[key]] returns another tree object (we have represented this by a dict object --> 
    that is if result is a dict object) we know that we have not arrived at a root node and have to run deeper the tree. 
    Okay... Look at your drawn tree in front of you... what are you doing?...well, you run down the next branch... 
    exactly as we have done it above with the slight difference that we already have passed a node and therewith 
    have to run only a fraction of the tree --> You clever guy! That "fraction of the tree" is exactly what we have stored
    under 'result'.
    So we simply call our predict method using the same query instance (we do not have to drop any features from the query
    instance since for instance the feature for the root node will not be available in any of the deeper sub_trees and hence 
    we will simply not find that feature) as well as the "reduced / sub_tree" stored in result.

    SUMMARIZED: If we have a query instance consisting of values for features, we take this features and check if the 
    name of the root node is equal to one of the query features.
    If this is true, we run down the root node outgoing branch whose value equals the value of query feature == the root node.
    If we find at the end of this branch a leaf node (not a dict object) we return this value (this is our prediction).
    If we instead find another node (== sub_tree == dict objct) we search in our query for the feature which equals the value 
    of that node. Next we look up the value of our query feature and run down the branch whose value is equal to the 
    query[key] == query feature value. And as you can see this is exactly the recursion we talked about
    with the important fact that for each node we run down the tree, we check only the nodes and branches which are 
    below this node and do not run the whole tree beginning at the root node 
    --> This is why we re-call the classification function with 'result'

In [ ]:
def predict(query,tree,default = 1):
    #1.
    for key in list(query.keys()):
        if key in list(tree.keys()):
            #2.
            try:
                result = tree[key][query[key]] 
            except:
                return default
  
            #3.
            result = tree[key][query[key]]
            #4.
            if isinstance(result,dict):
                return predict(query,result)

            else:
                return result


Test function to calculate accuracy on test dataset and also build confusion matrix.

In [ ]:

def train_test_split(dataset):
    boundary = int(0.8 * dataset.shape[0])
    training_data = dataset.iloc[:boundary].reset_index(drop=True)
    testing_data = dataset.iloc[boundary:].reset_index(drop=True)
    return training_data,testing_data
    
def test(data,tree):
    queries = data.iloc[:,:-1].to_dict(orient = "records")
    predicted = pd.DataFrame(columns=["predicted"]) 
    for i in range(len(data)):
        predicted.loc[i,"predicted"] = predict(queries[i],tree,1.0) 
    print('The prediction accuracy is: ',(np.sum(predicted["predicted"] == data['Recidivism - Return to Prison numeric'])/len(data))*100,'%')
    tp = np.sum((predicted["predicted"] == data['Recidivism - Return to Prison numeric']) & (data['Recidivism - Return to Prison numeric'] == 1))
    tn = np.sum((predicted["predicted"] == data['Recidivism - Return to Prison numeric']) & (data['Recidivism - Return to Prison numeric'] == 0))
    fp = np.sum((predicted["predicted"] != data['Recidivism - Return to Prison numeric']) & (data['Recidivism - Return to Prison numeric'] == 1))
    fn = np.sum((predicted["predicted"] != data['Recidivism - Return to Prison numeric']) & (data['Recidivism - Return to Prison numeric'] == 0))
    print([tp,fp])
    print([fn,tn])

#Random Forest

Random forests or random decision forests are an ensemble learning method for classification, regression and other tasks that operate by constructing a multitude of decision trees at training time and outputting the class that is the mode of the classes (classification) or mean/average prediction (regression) of the individual trees. Random decision forests correct for decision trees' habit of overfitting to their training set. Random forests generally outperform decision trees, but their accuracy is lower than gradient boosted trees. However, data characteristics can affect their performance.[1](https://https://en.wikipedia.org/)



In [ ]:
def random_forest(training_data,max_depth,number_of_random_features,number_of_trees):
    forest = []
    for i in range(number_of_trees):
        random_feature_index = list(np.random.choice(np.arange(0,11), number_of_random_features, replace=False))
        random_features = []
        for j in range(number_of_random_features):
            random_features.append(training_data.columns[random_feature_index[j]])
        tree = ID3(training_data,training_data,random_features,0,max_depth)
        forest.append(tree)
    return forest

def test_forest(data,forest):
    queries = data.iloc[:,:-1].to_dict(orient = "records")
    predicted = pd.DataFrame(columns=["predicted"]) 
    for i in range(len(data)):
        results = np.zeros(len(forest))
        for t in range(len(forest)):
            results[t] = predict(queries[i],forest[t],1.0)
        if (np.sum(results == 1) > np.sum(results == 0)):
            final = 1
        else:
            final = 0
        predicted.loc[i,"predicted"] = final
    print('The prediction accuracy is: ',(np.sum(predicted["predicted"] == data['Recidivism - Return to Prison numeric'])/len(data))*100,'%')
    tp = np.sum((predicted["predicted"] == data['Recidivism - Return to Prison numeric']) & (data['Recidivism - Return to Prison numeric'] == 1))
    tn = np.sum((predicted["predicted"] == data['Recidivism - Return to Prison numeric']) & (data['Recidivism - Return to Prison numeric'] == 0))
    fp = np.sum((predicted["predicted"] != data['Recidivism - Return to Prison numeric']) & (data['Recidivism - Return to Prison numeric'] == 1))
    fn = np.sum((predicted["predicted"] != data['Recidivism - Return to Prison numeric']) & (data['Recidivism - Return to Prison numeric'] == 0))
    print([tp,fp])
    print([fn,tn])

#Evaluation


In [ ]:
dataset = shuffle(dataset)
training_data = train_test_split(dataset)[0]
testing_data = train_test_split(dataset)[1] 
max_depth = 3
number_of_random_features = 3
number_of_trees = 21
forest = random_forest(training_data,max_depth,number_of_random_features,number_of_trees)
tree = ID3(training_data,training_data,training_data.columns[:-1],0,max_depth)
print('Structure of Tree:')
pprint(tree)
print('Accuracy and confusion matrix of tree:')
test(testing_data,tree)
print('Accuracy and confusion matrix of random forest:')
test_forest(testing_data,forest)

boundary = int(0.8 * len(dataset))
test_features = dataset.iloc[boundary:,:-1].values
test_targets = dataset.iloc[boundary:,-1].values
X = dataset.iloc[:boundary,:-1].values
y = dataset.iloc[:boundary,-1].values

for i in range(10):
    le = preprocessing.LabelEncoder()
    le.fit(X[:,i])
    X[:,i] = le.transform(X[:,i])
    test_features[:,i] = le.transform(test_features[:,i])
clf = RandomForestClassifier(criterion='entropy',max_depth=3, random_state=0)
clf.fit(X, y)
y_test = test_targets
y_pred = clf.predict(test_features)
print('Accuracy and confusion matrix of random forest using sci-kit library:')
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred)*100)

Structure of Tree:
{'Fiscal Year Released': {2010: {'Recidivism Reporting Year': {2013.0: {'Race - Ethnicity': {'Black': 1.0,
                                                                                             'White': 1.0}}}},
                          2013: {'Age At Release': {'<45': {'Main Supervising District': {'3JD': 1.0,
                                                                                          '5JD': 1.0}},
                                                    '>45': {'Convicting Offense Type': {'Drug': 1.0,
                                                                                        'Other': 1.0,
                                                                                        'Violent': 1.0}}}},
                          2015: {'Release Type': {'Discharged End of Sentence': {'Main Supervising District': {'3JD': 0.0,
                                                                                                               '5JD': 0.0}}

#Discussion

We can see that from testing random forset, we have a just little better performance than desicion tree or we just exprience equal performance. That's because "Fiscal Year Released" feature is determinative feature. By making desicion based on this attribute we almost done with predictions.